# keras CNN predict rectangle box

In [1]:
# !pip3 install tensorflow

In [2]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import pickle
from utils_misc import pickle_load_or_create

2024-07-24 14:04:15.350360: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-24 14:04:15.394786: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-24 14:04:16.231852: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
model = pickle_load_or_create('../models/rectangles_CNN_model_history', lambda: None, True)
print(model)

None


In [4]:
from DataGeneratorFrames import DataGeneratorRectangles

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
config = pickle_load_or_create('CNN_rectangles_3x3_rgb', lambda:{
    'convolution': (3,3),
    'dim':64,
    'rgb':True,
    'unique_labels': 3,
}, config=True)
config

{'convolution': (3, 3), 'dim': 64, 'rgb': True, 'unique_labels': 3}

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization, TimeDistributed, Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam

if model is None:
    model = Sequential()
    model.add(Conv2D(filters=16, kernel_size=config['convolution'],
                     input_shape=(config['dim'], config['dim'], 3 if config['rgb'] else 1)))
    model.add(MaxPool2D())
    model.add(BatchNormalization())
    
    model.add(Conv2D(filters=32, kernel_size=(3, 3)))
    model.add(BatchNormalization())
    
    model.add(Flatten())  # Flatten each frame
    model.add(Dense(64, activation='relu'))
    model.add(Dense(config['unique_labels'], activation='linear'))
    
    model.compile(optimizer=Adam(), 
                  loss='mean_absolute_error', 
                  metrics=['mean_absolute_error', 'mean_squared_error'])
else:
    model = model.model

/home/miked/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 31, 31, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 29, 29, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 26912)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     1,722,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,727,907 (6.59 MB)

 Trainable params: 1,727,811 (6.59 MB)

 Non-trainable params: 96 (384.00 B)

In [8]:
# Parameters
params = {'dim': (config['dim'],config['dim']),
          'n_classes': config['unique_labels'],
          'n_channels': 3 if config['rgb'] else 1,
          'shuffle': True,
}

training_generator = DataGeneratorRectangles(train=True, **params)
test_generator = DataGeneratorRectangles(train=False, **params)

connection established
DataGeneratorSkillBorders init done
on_epoch_end_called
connection established
DataGeneratorSkillBorders init done
on_epoch_end_called


In [9]:
training_generator.batch_order

,videoID,batch_nr_video,frame_amount,batch_id
0,4,54,32,0
1,7,21,32,1
2,4,24,32,2
3,7,31,32,3
4,4,32,32,4
...,...,...,...,...
310,6,24,32,310
311,5,2,32,311
312,1,17,32,312
313,4,58,32,313


In [10]:
%%time
X, y = training_generator.__getitem__(25)

 __getitem__(25)
CPU times: user 1.67 s, sys: 186 ms, total: 1.86 s
Wall time: 615 ms


In [11]:
X.shape

(32, 64, 64, 3)

In [12]:
y.shape

(32, 3)

In [13]:
y[:5]

array([[0.619444, 0.566146, 0.832972],
       [0.619444, 0.566146, 0.832972],
       [0.619444, 0.566146, 0.832972],
       [0.619444, 0.566146, 0.832972],
       [0.619444, 0.566146, 0.832972]])

In [ ]:
%%time
history = model.fit(training_generator, epochs=5, 
                    validation_data=test_generator, shuffle=False)

 __getitem__(0)
 __getitem__(1)
Epoch 1/5
 __getitem__(0)
 __getitem__(1)
  1/315 ━━━━━━━━━━━━━━━━━━━━ 2:18 440ms/step - loss: 0.0712 - mean_absolute_error: 0.0712 - mean_squared_error: 0.0135 __getitem__(2)
  2/315 ━━━━━━━━━━━━━━━━━━━━ 2:32 488ms/step - loss: 0.0747 - mean_absolute_error: 0.0747 - mean_squared_error: 0.0123 __getitem__(3)
  3/315 ━━━━━━━━━━━━━━━━━━━━ 2:24 464ms/step - loss: 0.0738 - mean_absolute_error: 0.0738 - mean_squared_error: 0.0113 __getitem__(4)
  4/315 ━━━━━━━━━━━━━━━━━━━━ 2:17 441ms/step - loss: 0.0705 - mean_absolute_error: 0.0705 - mean_squared_error: 0.0103 __getitem__(5)
  5/315 ━━━━━━━━━━━━━━━━━━━━ 2:19 449ms/step - loss: 0.0691 - mean_absolute_error: 0.0691 - mean_squared_error: 0.0097 __getitem__(6)
  6/315 ━━━━━━━━━━━━━━━━━━━━ 2:14 436ms/step - loss: 0.0673 - mean_absolute_error: 0.0673 - mean_squared_error: 0.0091 __getitem__(7)
  7/315 ━━━━━━━━━━━━━━━━━━━━ 2:13 435ms/step - loss: 0.0659 - mean_absolute_error: 0.0659 - mean_squared_error: 0.0086 __g

In [15]:
training_generator.batch_order

,videoID,batch_nr_video,frame_amount,batch_id
0,7,23,32,0
1,3,7,32,1
2,5,6,32,2
3,4,57,32,3
4,3,18,32,4
...,...,...,...,...
310,6,3,32,310
311,5,47,32,311
312,5,43,32,312
313,4,19,32,313


In [16]:
training_generator.__getitem__(4)

 __getitem__(4)


(array([[[[0.30196078, 0.32156863, 0.32156863],
          [0.30980392, 0.32156863, 0.32156863],
          [0.32156863, 0.33333333, 0.33333333],
          ...,
          [0.27058824, 0.28235294, 0.28235294],
          [0.2745098 , 0.28627451, 0.28627451],
          [0.26666667, 0.27843137, 0.27843137]],
 
         [[0.31372549, 0.3254902 , 0.3254902 ],
          [0.32941176, 0.34117647, 0.34117647],
          [0.3254902 , 0.3372549 , 0.3372549 ],
          ...,
          [0.29411765, 0.30588235, 0.30588235],
          [0.29411765, 0.30588235, 0.30588235],
          [0.27843137, 0.29019608, 0.29019608]],
 
         [[0.30980392, 0.32156863, 0.32156863],
          [0.30980392, 0.32156863, 0.32156863],
          [0.30196078, 0.32156863, 0.32156863],
          ...,
          [0.31372549, 0.31372549, 0.31372549],
          [0.28627451, 0.29803922, 0.29803922],
          [0.29019608, 0.30196078, 0.30196078]],
 
         ...,
 
         [[0.75686275, 0.74117647, 0.70196078],
          [0.69803

In [17]:
pd.DataFrame(history.history)

,loss,mean_absolute_error,mean_squared_error,val_loss,val_mean_absolute_error,val_mean_squared_error
0,2.638381,2.638381,74.932465,0.402151,0.402151,0.232742
1,0.318309,0.318309,0.160886,0.209560,0.209560,0.075239
2,0.140887,0.140887,0.042920,0.105161,0.105161,0.017849
3,0.080449,0.080449,0.016770,0.078056,0.078056,0.012907
4,0.077660,0.077660,0.015383,0.076103,0.076103,0.012393


In [18]:
model.fit?

Signature:
model.fit(
    x=None,
    y=None,
    batch_size=None,
    epochs=1,
    verbose='auto',
    callbacks=None,
    validation_split=0.0,
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
)
Docstring:
Trains the model for a fixed number of epochs (dataset iterations).

Args:
    x: Input data. It could be:
        - A NumPy array (or array-like), or a list of arrays
        (in case the model has multiple inputs).
        - A tensor, or a list of tensors
        (in case the model has multiple inputs).
        - A dict mapping input names to the corresponding array/tensors,
        if the model has named inputs.
        - A `tf.data.Dataset`. Should return a tuple
        of either `(inputs, targets)` or
        `(inputs, targets, sample_weights)`.
        - A `keras.utils.PyDataset` returning `(inputs,
      

In [19]:
# with open('../models/frames_skillborder_CNN_model_history.pkl', 'wb') as handle:
    # pickle.dump(history, handle)

In [20]:
X, y = test_generator.__getitem__(15)

 __getitem__(15)


In [21]:
model.predict(X), y

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step


(array([[0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0.94317406],
        [0.51121783, 0.54161423, 0